Packages

In [2]:
import torch
import numpy as np
import h5py
import dotenv
from dotenv import load_dotenv
import os
import random
print(os.getcwd())
os.chdir(os.path.split(os.getcwd())[0])
print(os.getcwd())
load_dotenv()

ModuleNotFoundError: No module named 'dotenv'

Read data

In [2]:
with h5py.File(".\\dataset.h5", 'r') as f:
  data = list(f["standardData"]) # read standardized data
  # data = list(f["unitData"]) # read unit normalized data

# dataset organized as list of 4 numpy.ndarray elements:
# x coordinates, y coordinates, time steps, pressure values.
# reorganize data in [nodeQuantity * time steps, 4] matrix with
# one information per column
dataMatrix = np.concatenate(([data[i][:, None] for i in range(4)]), axis = 1)
print(f"Dataset shape: {dataMatrix.shape}")

Dataset shape: (391036, 4)


Organize data

In [ ]:
# choose n_sensors random points in physical domain (which has 3371 nodes)
randNodeIDs = random.sample(range(3372), k = n_sensors)
# [sensor, case] matrix for input_dataset. Each line contains
# the pressure history of a specific "sensor" for the FIRST n_cases time steps.
input_dataset = np.concatenate(
    ([dataMatrix[randNodeIDs + np.tile(3371 * step, 1), 2][:, None] for step in range(n_cases)]),
    axis = 1
)
print(f"Input dataset shape: {input_dataset.shape}")

timeStepAmount = int(dataMatrix.shape[0] / 3371) # number of time steps
# same as input_dataset, but for the LAST n_cases time steps
output_dataset = np.concatenate(
    ([dataMatrix[
        randNodeIDs + np.tile(3371 * step, 1), 2
    ][:, None] for step in range(timeStepAmount - n_cases, timeStepAmount)]),
    axis = 1
)
print(f"Output dataset shape: {output_dataset.shape}")

General parameters

In [ ]:
n_cases = 30
n_sensors = 1000
branchDepth = 7 # number of layers in branch network (MLP architecture)
branchWidth = 100 # number of neurons in layers of branch network (MLP architecture)
trunkDepth = 7 # number of layers in trunk network (MLP architecture)
trunkWidth = 100 # number of neurons in layers of trunk network (MLP architecture)
load = -1.21 * 0.02

Setup model

Loss and optimizer

Train